In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# Helpful
import tqdm

# Visualization
import matplotlib.pyplot as plt

# My Methods
from src.utils.CRPS import *
from src.utils.data_split import *
from src.models.EMOS import *
from src.models.EMOS_global.EMOS_global_load_score import *
from src.models.EMOS_global.EMOS_global_load_model import *
import data.raw.load_data_raw as ldr
import data.processed.load_data_processed as ldp
import data.processed.load_data_processed_denormed as ldpd
from src.models.CRPS_baseline.CRPS_load import *

2023-05-29 15:34:03.929328: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/anaconda3/lib/
2023-05-29 15:34:03.929353: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


<IPython.core.display.Javascript object>

In [3]:
# Basics
import numpy as np
import keras.backend as K
import tensorflow as tf

# Norms
from scipy.stats import norm

# Helpful
from tqdm import tqdm

<IPython.core.display.Javascript object>

### 1. Load Raw Data

In [4]:
# Load train data for variables t2m and ws10
train_t2m_ws10_denormed = ldpd.load_data_t2m_ws10_train_proc_denorm()
train_t2m_denormed = train_t2m_ws10_denormed[0]
train_ws10_denormed = train_t2m_ws10_denormed[1]

# Load test data for variables t2m and ws10
test_t2m_ws10_denormed = ldpd.load_data_t2m_ws10_test_proc_denorm()
test_t2m_denormed = test_t2m_ws10_denormed[0]
test_ws10_denormed = test_t2m_ws10_denormed[1]

# Load train and test data for normed t2m:
train_t2m_normed = ldp.load_data_t2m_train_proc_norm()
test_t2m_normed = ldp.load_data_t2m_test_proc_norm()

<IPython.core.display.Javascript object>

###  2. Data Split

In [5]:
# Train-test split into lead_times for t2m
X_train_t2m_denormed, y_train_t2m_denormed = split_var_lead_one(train_t2m_denormed)
X_test_t2m_denormed, y_test_t2m_denormed = split_var_lead_one(test_t2m_denormed)
# Train-test split into lead_times for ws10
X_train_ws10_denormed, y_train_ws10_denormed = split_var_lead_one(train_ws10_denormed)
X_test_ws10_denormed, y_test_ws10_denormed = split_var_lead_one(test_ws10_denormed)

# Train-test split into lead_times for t2m normed
X_train_t2m_normed, y_train_t2m_normed = split_var_lead_one(train_t2m_normed)
X_test_t2m_normed, y_test_t2m_normed = split_var_lead_one(test_t2m_normed)

<IPython.core.display.Javascript object>

Test with z500 denormed, to see whether it also leads to bad results:

In [6]:
train_z500_denormed = ldpd.load_data_z500_train_proc_denorm()
X_train_z500_denormed, y_train_z500_denormed = split_var_lead_one(train_z500_denormed)
train_u10_denormed = ldpd.load_data_u10_train_proc_denorm()
X_train_u10_denormed, y_train_u10_denormed = split_var_lead_one(train_u10_denormed)
train_v10_denormed = ldpd.load_data_v10_train_proc_denorm()
X_train_v10_denormed, y_train_v10_denormed = split_var_lead_one(train_v10_denormed)
train_t850_denormed = ldpd.load_data_t850_train_proc_denorm()
X_train_t850_denormed, y_train_t850_denormed = split_var_lead_one(train_t850_denormed)

<IPython.core.display.Javascript object>

### 3. Train EMOS model

##### Checking that crps_cost_function and crps_normal are equivalent:

In [11]:
crps_cost_function(
    y_test_t2m_denormed[0].values.flatten(),
    np.column_stack(
        (
            X_test_t2m_denormed[0].isel(mean_std=0).values.flatten(),
            X_test_t2m_denormed[0].isel(mean_std=1).values.flatten(),
        )
    ),
    theano=True,
)

<tf.Tensor: shape=(), dtype=float32, numpy=0.5084816>

<IPython.core.display.Javascript object>

In [12]:
crps_normal(
    mu=X_test_t2m_denormed[0].isel(mean_std=0).values.flatten(),
    sigma=X_test_t2m_denormed[0].isel(mean_std=1).values.flatten(),
    y=y_test_t2m_denormed[0].values.flatten(),
).mean()

0.5084816639136494

<IPython.core.display.Javascript object>

#### 3.1 Loading baseline to compare

In [57]:
ws10_crps_baseline = crps_load_lead_lat_lon("ws10")
t2m_crps_baseline = crps_load_lead_lat_lon("t2m")

<IPython.core.display.Javascript object>

In [59]:
print(f"Wind baseline: {ws10_crps_baseline[0].mean()}")
print(f"T2m baseline: {t2m_crps_baseline[0].mean()}")

Wind baseline: 0.7447009086608887
T2m baseline: 0.9354666921537256


<IPython.core.display.Javascript object>

### 3. Load EMOS global scores

In [4]:
EMOS_glob_scores_t2m = EMOS_global_load_score("t2m")
EMOS_glob_scores_ws10 = EMOS_global_load_score("ws10")

<IPython.core.display.Javascript object>

### 4. Load EMOS global models:

In [6]:
EMOS_glob_models_t2m = EMOS_global_load_model_t2m()

<IPython.core.display.Javascript object>

In [8]:
len(EMOS_glob_models_t2m)

31

<IPython.core.display.Javascript object>

In [11]:
EMOS_glob_models_t2m[20].get_weights()

[array([[0.9957687]], dtype=float32),
 array([1.5915295], dtype=float32),
 array([[1.1350194]], dtype=float32),
 array([0.426182], dtype=float32)]

<IPython.core.display.Javascript object>